<a href="https://colab.research.google.com/github/Parishri07/Medical-Imaging-using-PyTorch/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dicom2nifti

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 52.7 MB/s eta 0:00:00


In [2]:
import dicom2nifti

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path_to_dicom = "/content/drive/MyDrive/Colab Notebooks/Brain MRI scan/ST000001/SE000002"

In [5]:
output_path = "/content/sample_data"

dicom2nifti.convert_directory(path_to_dicom, output_path)

In [6]:
%matplotlib notebook
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

In [7]:
brain_mri = nib.load('/content/sample_data/301_dwi_anonymized.nii.gz')

In [8]:
brain_mri_data = brain_mri.get_fdata()

In [9]:
affine = brain_mri.affine
shape = brain_mri.shape

In [10]:
print(affine)

[[-8.14996898e-01  1.85306575e-02  2.94511348e-01  1.21936066e+02]
 [ 2.05970202e-02  8.14844251e-01  3.09533834e-01 -1.12818481e+02]
 [ 3.90408374e-02 -4.30558622e-02  5.98476839e+00 -6.33279419e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
shape

(288, 288, 25, 5)

In [30]:
selected_channel = 1  # Choose the desired channel (0-indexed)
brain_mri_reduced = brain_mri.slicer[:,:,:,selected_channel]

In [31]:
brain_mri_reduced.header.get_zooms()

(0.8161914, 0.8161914, 6.0)

In [32]:
brain_mri_data_reduced = brain_mri_reduced.get_fdata()

In [37]:
brain_mri_data_reduced.shape

(288, 288, 25)

Common axis codes are:

L: Left

R: Right

A: Anterior (front)

P: Posterior (back)

I: Inferior (bottom)

S: Superior (top)

In [25]:
nib.aff2axcodes(affine)

('L', 'A', 'S')

The first axis (x-axis) points to the Left.

The second axis (y-axis) points Anteriorly (front).

The third axis (z-axis) points Superiorly (top).

In [40]:
fig, axis = plt.subplots(1, 2)
axis[0].imshow(brain_mri_data_reduced[0, :, :], cmap='gray')
axis[1].imshow(brain_mri_data_reduced[0, :, :], cmap='gray')

**Convert from Physical to Voxel coordinates and vice versa**

In [42]:
voxel_coords = np.array((0, 0, 0, 1))
physical_coords = affine @ voxel_coords
physical_coords

array([ 121.93606567, -112.81848145,  -63.32794189,    1.        ])

In [43]:
affine[:, 3:]

array([[ 121.93606567],
       [-112.81848145],
       [ -63.32794189],
       [   1.        ]])

In [44]:
voxel_coords_manual = np.array((0, 0, 0))
physical_coords_manual = affine[:3, 3] + affine[:3, :3] @ voxel_coords_manual
physical_coords_manual

array([ 121.93606567, -112.81848145,  -63.32794189])

In [45]:
physical_coords = [121.93606567, -112.81848145,  -63.32794189,    1.]
(np.linalg.inv(affine) @ physical_coords).round()

array([ 0., -0.,  0.,  1.])

**Resampling**

In [47]:
print(brain_mri_reduced.shape)
print(brain_mri_reduced.header.get_zooms())

(288, 288, 25)
(0.8161914, 0.8161914, 6.0)


In [48]:
 import nibabel.processing

In [49]:
voxel_size = (2, 2, 2)
brain_mri_resized = nibabel.processing.conform(brain_mri_reduced, (128, 128, 100), voxel_size, orientation='LAS')
brain_mri_resized_data = brain_mri_resized.get_fdata()

In [50]:
print(brain_mri_resized.shape)
print(brain_mri_resized.header.get_zooms())

(128, 128, 100)
(2.0, 2.0, 2.0)


In [51]:
fig, axis = plt.subplots(1, 2)
axis[1].imshow(brain_mri_resized_data[:, :, 50], cmap='gray')

Normalization and Standardization

In [52]:
path_to_dicom = "/content/drive/MyDrive/Colab Notebooks/CT Medical images/dicom_dir"

In [55]:
output_path = "/content"

dicom2nifti.convert_directory(path_to_dicom, output_path)

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/dicom2nifti/convert_dir.py", line 85, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/usr/local/lib/python3.10/dist-packages/dicom2nifti/convert_dicom.py", line 118, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/usr/local/lib/python3.10/dist-packages/dicom2nifti/convert_generic.py", line 230, in dicom_to_nifti
    common.validate_slicecount(dicom_input)
  File "/usr/local/lib/python3.10/dist-packages/dicom2nifti/common.py", line 1023, in validate_slicecount
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/dicom2nifti/convert_dir.py", line 85, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_fil

In [ ]:
lung_ct = nib.load("lung_043.nii.gz")
lung_ct_data = lung_ct.get_fdata()

In [ ]:
lung_ct_data_standardized = lung_ct_data / 3071

In [ ]:
plt.figure()
plt.imshow(np.rot90(lung_ct_data_standardized[:, :, 50]), cmap="gray")

In [ ]:
lung_ct_lung_window = np.clip(lung_ct_data, -1000, -500)
plt.figure()
plt.imshow(np.rot90(lung_ct_lung_window[:, :, 50]), cmap="bone")

In [ ]:
lung_ct_soft_tissue_window = np.clip(lung_ct_data, -250, 250)
plt.figure()
plt.imshow(np.rot90(lung_ct_soft_tissue_window[:, :, 5]), cmap="gray")

In [ ]:
cardiac_mri = nib.load("la_003.nii.gz")
cardiac_mri_data = cardiac_mri.get_fdata()

In [ ]:
mean, std = cardiac_mri_data.mean(), cardiac_mri_data.std()
cardiac_mri_norm = (cardiac_mri_data - mean) / std
cardiac_mri_standardized = (cardiac_mri_norm - np.min(cardiac_mri_norm)) / (np.max(cardiac_mri_norm) - np.min(cardiac_mri_norm))

In [ ]:
plt.figure()
plt.imshow(cardiac_mri_standardized[:, :, 30], cmap="gray")